In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/PyAI'

/content/drive/MyDrive/PyAI


In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
train = datasets.MNIST(root = "/data", train = True, transform=transforms.ToTensor(), download = True)
test = datasets.MNIST(root = "/data", train = False, transform=transforms.ToTensor(), download = True)

train_data = DataLoader(train, batch_size = 100, shuffle = True)
test_data = DataLoader(test, batch_size = 1000, shuffle = False)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.55MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.22MB/s]


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#x, t 입력
#dataloader에서 처리

#함수 정의
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5), #(1*28*28) -> (16*24*24)
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5), #(16*24*24) -> (32*20*20)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32*20*20) -> (32*10*10)
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5), #(32*10*10) -> (64*6*6)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64*6*6) -> (64*3*3)
        )
        self.g = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10),
        )
    def forward(self,x) :
        x = self.f(x)
        x = x.reshape(-1, 64*3*3)
        x = self.g(x)
        return x
F = NN()
F = F.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.002)
epoch = 10
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_data :
#순전파
        y = F(x.to(device))
#손실함수
        loss = loss_func(y, t.to(device))
        loss_sum += loss
#역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(train_data)

    correct = 0
    total = 0
    for x, t in test_data :
        with torch.no_grad() :
            y = F(x.to(device))
        correct += (y.argmax(dim = 1) == t.to(device)).sum().item()
        total += len(x)
    acc = correct / total

    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc}")
    if (cnt >= 3) or (acc >= 0.99) :
        print("train halted")
        break

epoch 1 | loss 0.16232287883758545 | acc 0.9866
epoch 2 | loss 0.046882644295692444 | acc 0.9898
epoch 3 | loss 0.032998330891132355 | acc 0.9901
train halted


In [6]:
cnt = 0;
total = 0
err = []

for x, t in test_data :
    y = F(x.to(device))
    for i in range(x.shape[0]) :
        if torch.argmax(y[i]) == t.to(device)[i] :
            cnt += 1
        else :
            err.append(total)
        total += 1

print("correct_posibility : %f" %(cnt / total))
print(err)
torch.save(F.to("cpu"),"MNIST.pt")

correct_posibility : 0.990100
[62, 320, 321, 445, 449, 450, 582, 619, 625, 726, 740, 883, 938, 947, 1014, 1039, 1226, 1247, 1260, 1290, 1319, 1621, 1709, 1901, 1982, 2043, 2129, 2130, 2135, 2182, 2293, 2308, 2462, 2496, 2597, 2654, 2742, 2770, 2896, 2927, 2939, 2953, 2959, 2995, 3005, 3023, 3108, 3225, 3330, 3422, 3441, 3451, 3520, 3534, 3558, 3681, 3808, 3850, 3853, 3906, 3995, 4078, 4163, 4201, 4284, 4380, 4393, 4487, 4639, 4699, 4761, 4807, 5265, 5573, 5634, 5654, 6173, 6560, 6571, 6576, 6597, 6625, 6651, 6783, 7216, 7574, 8316, 8520, 8527, 9071, 9505, 9620, 9642, 9664, 9669, 9679, 9692, 9729, 9856]
